In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import json
import itertools
from tqdm import tqdm

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps+1)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [3]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [4]:
import numpy as np
from sklearn.linear_model import Ridge


def sigmoid(x: np.ndarray) -> np.ndarray:
    """Numerically stable logistic function."""
    return 1.0 / (1.0 + np.exp(-x))


class SwirlGatedMultiCycleESN:
    """
    Swirl-Gated k-Cycle Echo-State Network (SG-kC-ESN).

    Parameters
    ----------
    reservoir_size : int
        Total number of neurons N (must be divisible by n_cycles).
    n_cycles       : int, ≥ 2
        How many simple cycles to create.
    cycle_weight   : float
        Weight r on each ring edge.
    bridge_weight  : float
        Weight s on the k sparse bridges (one per cycle).
    input_scale    : float
        Scaling of random input matrix W_in.
    leak_rate      : float in (0,1]
        leaky-integrator update; 1 recovers standard ESN.
    ridge_alpha    : float
        ℓ₂ penalty used in the ridge read-out.
    swirl_beta, swirl_frequency, swirl_sigmoid
        Parameters of the static per-neuron swirl gate
            g_k = σ[β sin(ω·q + φ_c)]      if swirl_sigmoid
                  β sin(ω·q + φ_c)         otherwise
        with φ_c = 2πc / k   (c = ring id).
    """

    # ------------------------------- init ------------------------------ #
    def __init__(
        self,
        reservoir_size: int = 600,
        n_cycles: int = 4,
        cycle_weight: float = 0.9,
        bridge_weight: float = 0.25,
        input_scale: float = 0.5,
        leak_rate: float = 1.0,
        ridge_alpha: float = 1e-6,
        swirl_beta: float = 2.0,
        swirl_frequency: float | None = None,
        swirl_sigmoid: bool = True,
        seed: int = 42,
        use_polynomial_readout: bool = True,
    ):
        if n_cycles < 2:
            raise ValueError("n_cycles must be at least 2")
        if reservoir_size % n_cycles:
            raise ValueError("reservoir_size must be divisible by n_cycles")

        # -------------- basic bookkeeping --------------------------------
        self.N = reservoir_size
        self.k = n_cycles
        self.m = reservoir_size // n_cycles          # neurons per ring

        # -------------- hyper-parameters ---------------------------------
        self.r = cycle_weight
        self.s = bridge_weight
        self.input_scale = input_scale
        self.alpha = leak_rate
        self.ridge_alpha = ridge_alpha
        self.beta = swirl_beta
        self.omega = (
            swirl_frequency if swirl_frequency is not None else 2.0 * np.pi / self.m
        )
        self.swirl_sigmoid = swirl_sigmoid
        self.seed = seed
        self.use_poly = use_polynomial_readout

        # -------------- placeholders to be filled ------------------------
        self.W_res: np.ndarray | None = None
        self.W_in: np.ndarray | None = None
        self.W_out: np.ndarray | None = None
        self.gate: np.ndarray | None = None
        self.x = np.zeros(self.N, dtype=np.float32)

        # -------------- one-off construction -----------------------------
        self._build_reservoir()
        self._build_swirl_gate()

    # =========================== builders =============================== #
    def _build_reservoir(self):
        """Construct the k-cycle recurrent matrix W_res (shape N × N)."""
        m, r, s, k = self.m, self.r, self.s, self.k

        # 1) ring block C_r : unidirectional permutation matrix scaled by r
        C_r = np.zeros((m, m), dtype=np.float32)
        for i in range(m):
            C_r[(i + 1) % m, i] = r

        # 2) bridge block S : rank-1 matrix with single non-zero entry (0,0)
        S = np.zeros((m, m), dtype=np.float32)
        S[0, 0] = s

        # 3) assemble full block matrix
        W = np.zeros((self.N, self.N), dtype=np.float32)

        def put_block(row_ring: int, col_ring: int, block: np.ndarray):
            i0, j0 = row_ring * m, col_ring * m
            W[i0 : i0 + m, j0 : j0 + m] = block

        for c in range(k):
            put_block(c, c, C_r)                   # diagonal ring
            put_block(c, (c - 1) % k, S)           # bridge to predecessor

        self.W_res = W

    def _build_swirl_gate(self):
        """Pre-compute static gain vector g (length N)."""
        g = np.empty(self.N, dtype=np.float32)
        for k_idx in range(self.N):
            ring_id = k_idx // self.m
            local_q = k_idx % self.m
            phi_c = 2.0 * np.pi * ring_id / self.k
            raw = self.beta * np.sin(self.omega * local_q + phi_c)
            g[k_idx] = sigmoid(raw) if self.swirl_sigmoid else raw
        self.gate = g

    # ====================== low-level reservoir ops ===================== #
    def _apply_gate(self, vec: np.ndarray) -> np.ndarray:
        return self.gate * vec

    def _update_state(self, u_t: np.ndarray):
        """Single ESN step with optional leakage."""
        pre = self.W_res @ self.x + self.W_in @ u_t
        gated = self._apply_gate(pre)
        new_x = np.tanh(gated)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * new_x

    def reset_state(self):
        self.x.fill(0.0)

    # ====================== read-out training (ridge) =================== #
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher forcing pass • inputs [T, d_in] → states, then fit ridge.
        """
        T, d_in = inputs.shape
        if T <= discard + 1:
            raise ValueError("Not enough data for training")

        rng = np.random.default_rng(self.seed)
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.N, d_in)) * self.input_scale
        ).astype(np.float32)

        self.reset_state()
        states = []
        for t in range(T):
            self._update_state(inputs[t])
            if t >= discard:
                states.append(self.x.copy())

        states = np.asarray(states, dtype=np.float32)          # [T-discard, N]
        Y = targets[discard:]                                  # same length

        if self.use_poly:
            feats = np.concatenate(
                [states, states * states, np.ones((states.shape[0], 1), dtype=np.float32)],
                axis=1,
            )
        else:
            feats = states

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(feats, Y)
        self.W_out = reg.coef_.astype(np.float32)             # d_out × feat_dim

    # ======================== autoregressive roll-out =================== #
    def predict_autoregressive(
        self, initial_input: np.ndarray, n_steps: int
    ) -> np.ndarray:
        if self.W_out is None:
            raise RuntimeError("Call fit_readout() before prediction.")

        d_in = initial_input.shape[0]
        preds = np.empty((n_steps, self.W_out.shape[0]), dtype=np.float32)

        #self.reset_state()
        current_in = initial_input.astype(np.float32).copy()

        for t in range(n_steps):
            self._update_state(current_in)

            if self.use_poly:
                big_x = np.concatenate(
                    [self.x, self.x * self.x, np.ones(1, dtype=np.float32)]
                )
            else:
                big_x = self.x

            y_t = (self.W_out @ big_x).astype(np.float32)
            preds[t] = y_t
            current_in = y_t[:d_in]  # feedback: assume d_in ≤ d_out

        return preds


In [5]:
# grid={
#     "cycle_weight" : [0.95],
#     "bridge_weight" : [0.4],
#     "input_scale" : [0.7],
#     "leak_rate" :[0.9],
#     "ridge_alpha": [1e-6],
#     "swirl_beta" : [2.2],
#     "swirl_sigmoid" : [True],
#     "use_polynomial_readout": [True]
# }

In [6]:
grid={
    "cycle_weight" : [0.4],
    "bridge_weight" : [0.1],
    "input_scale" : [2],
    "leak_rate" :[0.8],
    "ridge_alpha": [1e-6],
    "swirl_beta" : [3],
    "swirl_sigmoid" : [True],
    "use_polynomial_readout": [True]
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    # Precompute param combinations
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # tqdm adds a progress bar for better visualization
    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores = []
        
        # Run all 20 seeds
        for initial_state in [[1.0,1.0,1.0],[1.0,2.0,3.0],[2.0,1.5,4.0]]:
            tmax = 250
            dt   = 0.02
            t_vals, rossler_traj = generate_lorenz_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )
            
            washout = 2000
            t_vals = t_vals[washout:]
            rossler_traj = rossler_traj[washout:]
            
            # normalize
            scaler = MinMaxScaler()
            scaler.fit(rossler_traj)
            rossler_traj = scaler.transform(rossler_traj)
            
            T_data = len(rossler_traj)
            for train_frac in [0.7,0.75,0.8]:
                train_end = int(train_frac*(T_data-1))
                train_input  = rossler_traj[:train_end]
                train_target = rossler_traj[1:train_end+1]
                test_input   = rossler_traj[train_end:-1]
                test_target  = rossler_traj[train_end+1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]
                for seed in np.arange(1,6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    _, _, T_VPT_s = compute_valid_prediction_time(test_target,preds,t_vals,0.4,0.9056,dt)
                    seed_scores.append(T_VPT_s)
        mean_score = float(np.mean(seed_scores))
        std_dev    = float(np.std(seed_scores))
        # is_stable  = std_dev < 1.5
        # status     = "Stable" if is_stable else "Unstable"
        
        # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
        #       f"Std Dev={std_dev:.3f} → {status}")

        results.append({
            "params":      params,
            "seed_scores": seed_scores,
            "mean_T_VPT":  mean_score,
            "std_dev":     std_dev,
            # "stable":      is_stable
        })

    # Save results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")
    
    return results

In [8]:
run_grid_search(SwirlGatedMultiCycleESN, grid, "MPPR", output_path="mppr 3.json")


== Initial grid search for MPPR with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [05:34<00:00, 334.97s/it]


All results saved to `mppr 3.json`


[{'params': {'cycle_weight': 0.4,
   'bridge_weight': 0.1,
   'input_scale': 2,
   'leak_rate': 0.8,
   'ridge_alpha': 1e-06,
   'swirl_beta': 3,
   'swirl_sigmoid': True,
   'use_polynomial_readout': True},
  'seed_scores': [np.float64(7.625152000000001),
   np.float64(7.6975999999999996),
   np.float64(7.661376000000001),
   np.float64(7.679488000000003),
   np.float64(7.715712000000003),
   np.float64(7.842496000000003),
   np.float64(7.987392),
   np.float64(7.842496000000003),
   np.float64(7.860607999999999),
   np.float64(7.878720000000002),
   np.float64(8.711872000000003),
   np.float64(8.820544000000002),
   np.float64(8.784320000000003),
   np.float64(8.766207999999999),
   np.float64(8.838655999999997),
   np.float64(7.969280000000003),
   np.float64(7.969280000000003),
   np.float64(7.933055999999998),
   np.float64(7.951168000000001),
   np.float64(8.023615999999999),
   np.float64(8.512639999999998),
   np.float64(8.6032),
   np.float64(8.548863999999998),
   np.float64(